In [1]:
import csv
from datetime import datetime
import os
import re
import unicodedata

#### Setting variables

In [2]:
path_wd   = os.getcwd()
path_root = path_wd.rsplit("\\",0)[0]


path_raw = path_root + '\\raw\\'

path_raw_bancos = path_raw + 'bancos\\'
path_raw_empregados = path_raw + 'empregados\\'
path_raw_reclamacoes = path_raw + 'reclamacoes\\'

flist_raw_bancos = os.listdir(path_raw_bancos)
flist_raw_empregados = os.listdir(path_raw_empregados)
flist_raw_reclamacoes = os.listdir(path_raw_reclamacoes)


path_stage = path_root + '\\stage\\'

path_stage_bancos = path_stage + 'bancos\\'
path_stage_empregados = path_stage + 'empregados\\'
path_stage_reclamacoes = path_stage + 'reclamacoes\\'

#### Data Extraction and Cleansing

In [3]:
class DataFrame:
    def __init__(self, data, columns=None):
        self.data = data
        self.columns = columns
    
    def __repr__(self):
        return str(self.data)

    def rename_column(self, old_name, new_name):
        if old_name in self.columns:
            column_index = self.columns.index(old_name)
            self.columns[column_index] = new_name

    def handle_missing_values(self, column_name, handle_func):
        column_index = self.columns.index(column_name)
        for row in self.data:
            if row[column_index] is None or row[column_index] == "" or row[column_index] == " ":
                row[column_index] = handle_func(row[column_index])

    def cast_column_to_int(self, column_name):
        column_index = self.columns.index(column_name)
        for row in self.data:
            try:
                row[column_index] = int(row[column_index])
            except ValueError:
                pass

    def apply_function(self, column_name, func):
        column_index = self.columns.index(column_name)
        for row in self.data:
            row[column_index] = func(row[column_index])

    def to_csv(self, output_file, delimiter=";"):
        with open(output_file, "w", newline="") as csvfile:
            writer = csv.writer(csvfile, delimiter=delimiter)
            writer.writerow(self.columns)
            for row in self.data:
                writer.writerow(row)

    

In [4]:
def get_df(fname, delimiter, header, charset):
    with open(fname, "r", encoding=charset) as csvfile:
        reader = csv.reader(csvfile, delimiter=delimiter)
        
        if header:
            header_row = next(reader)

        data_rows = [row for row in reader]

    if header:
        df = DataFrame(data_rows, columns=header_row)
    else:
        df = DataFrame(data_rows)

    return df

In [5]:
def remove_accented_characters(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

In [6]:
def handle_missing(value):
    if value is None or value == "" or value == " ":
        return "NULL"
    return value

Bancos

In [7]:
fname_bancos = path_raw_bancos + flist_raw_bancos[0]
delimiter = ','
header = True
charset = 'utf-8'

column_to_rename_old = ['\ufeffSegmento','CNPJ','Nome']
column_to_rename_new = ['segmento','cnpj','nome']

column_to_clean = 'nome'

# Load df
df_bancos = get_df(fname_bancos, delimiter, header, charset)

# Apply column rename func
for i in range(len(column_to_rename_old)):
    df_bancos.rename_column(column_to_rename_old[i], column_to_rename_new[i])

# Apply remove accent char func 
df_bancos.apply_function(column_to_clean, remove_accented_characters)

# Apply cast CNPJ column to int
df_bancos.cast_column_to_int("cnpj")

# Apply None to missing values
df_bancos.handle_missing_values("cnpj", handle_missing)

Empregados

In [8]:
fname_empregados_01 = path_raw_empregados + flist_raw_empregados[0]
fname_empregados_02 = path_raw_empregados + flist_raw_empregados[1]
delimiter = '|'
header = True
charset = 'utf-8'

column_to_rename_old = ['\ufeffemployer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer-website', 'employer-headquarters', 'employer-founded', 'employer-industry', 'employer-revenue', 'url', 'Geral', 'Cultura e valores', 'Diversidade e inclusÃ£o', 'Qualidade de vida', 'Alta lideranÃ§a', 'RemuneraÃ§Ã£o e benefÃ\xadcios', 'Oportunidades de carreira', 'Recomendam para outras pessoas(%)', 'Perspectiva positiva da empresa(%)', 'CNPJ', 'Nome', 'match_percent']
column_to_rename_new = ['employer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer_website', 'employer_headquarters', 'employer_founded', 'employer_industry', 'employer_revenue', 'url', 'geral', 'cultura_e_valores', 'diversidade_e_inclusao', 'qualidade_de_vida', 'alta_lideranca', 'remuneracao_e_beneficios', 'oportunidades_de_carreira', 'recomendam_para_outras_pessoas_perc', 'perspectiva_positiva_da_empresa_perc', 'cnpj', 'nome', 'match_percent']

column_to_clean = ['employer_name', 'nome']

# Load dfs
df_empregados_01 = get_df(fname_empregados_01, delimiter, header, charset)
df_empregados_02 = get_df(fname_empregados_02, delimiter, header, charset)


# Apply column rename func
for i in range(len(column_to_rename_old)):
    df_empregados_01.rename_column(column_to_rename_old[i], column_to_rename_new[i])
    df_empregados_02.rename_column(column_to_rename_old[i], column_to_rename_new[i])

# Apply remove accent char func
for i in range(len(column_to_clean)):
    df_empregados_01.apply_function(column_to_clean[i], remove_accented_characters)
    df_empregados_02.apply_function(column_to_clean[i], remove_accented_characters)

# Apply cast CNPJ column to int
df_empregados_01.cast_column_to_int("cnpj")

# Apply None to missing values
df_empregados_01.handle_missing_values("cnpj", handle_missing)

Reclamações

In [9]:
fname_reclamacoes_202101 = path_raw_reclamacoes + flist_raw_reclamacoes[0]
fname_reclamacoes_202102 = path_raw_reclamacoes + flist_raw_reclamacoes[1]
fname_reclamacoes_202103 = path_raw_reclamacoes + flist_raw_reclamacoes[2]
fname_reclamacoes_202104 = path_raw_reclamacoes + flist_raw_reclamacoes[3]
fname_reclamacoes_202201 = path_raw_reclamacoes + flist_raw_reclamacoes[4]
fname_reclamacoes_202202 = path_raw_reclamacoes + flist_raw_reclamacoes[5]
fname_reclamacoes_202203 = path_raw_reclamacoes + flist_raw_reclamacoes[6]
fname_reclamacoes_202204 = path_raw_reclamacoes + flist_raw_reclamacoes[7]

delimiter = ';'
header = True
charset = 'utf-8'

column_to_rename_old = ['\ufeffAno', 'Trimestre', 'Categoria', 'Tipo', 'CNPJ IF', 'Instituição financeira', 'Índice', 'Quantidade de reclamações reguladas procedentes', 'Quantidade de reclamações reguladas - outras', 'Quantidade de reclamações não reguladas', 'Quantidade total de reclamações', 'Quantidade total de clientes – CCS e SCR', 'Quantidade de clientes – CCS', 'Quantidade de clientes – SCR', ',,']
column_to_rename_new = ['ano', 'trimestre', 'categoria', 'tipo', 'cnpj_if', 'instituicao_financeira', 'indice', 'quantidade_de_reclamacoes_reguladas_procedentes', 'quantidade_de_reclamacoes_reguladas_outras', 'quantidade_de_reclamacoes_nao_reguladas', 'quantidade_total_de_reclamacoes', 'quantidade_total_de_clientes_ccs_e_scr', 'quantidade_de_clientes_ccs', 'quantidade_de_clientes_scr', ',,']

column_to_clean = 'instituicao_financeira'

# Load dfs
df_reclamacoes_202101 = get_df(fname_reclamacoes_202101, delimiter, header, charset)
df_reclamacoes_202102 = get_df(fname_reclamacoes_202102, delimiter, header, charset)
df_reclamacoes_202103 = get_df(fname_reclamacoes_202103, delimiter, header, charset)
df_reclamacoes_202104 = get_df(fname_reclamacoes_202104, delimiter, header, charset)
df_reclamacoes_202201 = get_df(fname_reclamacoes_202201, delimiter, header, charset)
df_reclamacoes_202202 = get_df(fname_reclamacoes_202202, delimiter, header, charset)
df_reclamacoes_202203 = get_df(fname_reclamacoes_202203, delimiter, header, charset)
df_reclamacoes_202204 = get_df(fname_reclamacoes_202204, delimiter, header, charset)


# Apply column rename func
for i in range(len(column_to_rename_old)):
    df_reclamacoes_202101.rename_column(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202102.rename_column(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202103.rename_column(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202104.rename_column(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202201.rename_column(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202203.rename_column(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202204.rename_column(column_to_rename_old[i], column_to_rename_new[i])

# Apply remove accent char func
df_reclamacoes_202101.apply_function(column_to_clean, remove_accented_characters)
df_reclamacoes_202102.apply_function(column_to_clean, remove_accented_characters)
df_reclamacoes_202103.apply_function(column_to_clean, remove_accented_characters)
df_reclamacoes_202104.apply_function(column_to_clean, remove_accented_characters)
df_reclamacoes_202201.apply_function(column_to_clean, remove_accented_characters)
df_reclamacoes_202203.apply_function(column_to_clean, remove_accented_characters)
df_reclamacoes_202204.apply_function(column_to_clean, remove_accented_characters)

# Apply cast CNPJ column to int
df_reclamacoes_202101.cast_column_to_int("cnpj_if")
df_reclamacoes_202102.cast_column_to_int("cnpj_if")
df_reclamacoes_202103.cast_column_to_int("cnpj_if")
df_reclamacoes_202104.cast_column_to_int("cnpj_if")
df_reclamacoes_202201.cast_column_to_int("cnpj_if")
df_reclamacoes_202203.cast_column_to_int("cnpj_if")
df_reclamacoes_202204.cast_column_to_int("cnpj_if")

# Apply None to missing values
df_reclamacoes_202101.handle_missing_values("cnpj_if", handle_missing)
df_reclamacoes_202102.handle_missing_values("cnpj_if", handle_missing)
df_reclamacoes_202103.handle_missing_values("cnpj_if", handle_missing)
df_reclamacoes_202104.handle_missing_values("cnpj_if", handle_missing)
df_reclamacoes_202201.handle_missing_values("cnpj_if", handle_missing)
df_reclamacoes_202203.handle_missing_values("cnpj_if", handle_missing)
df_reclamacoes_202204.handle_missing_values("cnpj_if", handle_missing)

#### Output

In [10]:
# Bancos
# nome do arquivo
fname_bancos = path_stage_bancos + flist_raw_bancos[0]
fname_bancos = fname_bancos.replace('.csv', '_cleaned.csv')
fname_bancos = fname_bancos.replace('\\', '/')

# write CSV
df_bancos.to_csv(fname_bancos)
print(fname_bancos)

c:/Users/Diane/git/grupo1-ingestao-dados/stage/bancos/EnquadramentoInicia_v2_cleaned.csv


In [11]:
# Empregados
# nome do arquivo
fname_empregados_01 = path_stage_empregados + flist_raw_empregados[0]
fname_empregados_01 = fname_empregados_01.replace('.csv', '_cleaned.csv')
fname_empregados_01 = fname_empregados_01.replace('\\', '/')

fname_empregados_02 = path_stage_empregados + flist_raw_empregados[1]
fname_empregados_02 = fname_empregados_02.replace('.csv', '_cleaned.csv')
fname_empregados_02 = fname_empregados_02.replace('\\', '/')

# write CSV
df_empregados_01.to_csv(fname_empregados_01)
df_empregados_02.to_csv(fname_empregados_02)

print(fname_empregados_01)
print(fname_empregados_02)

c:/Users/Diane/git/grupo1-ingestao-dados/stage/empregados/glassdoor_consolidado_join_match_less_v2_cleaned.csv
c:/Users/Diane/git/grupo1-ingestao-dados/stage/empregados/glassdoor_consolidado_join_match_v2_cleaned.csv


In [12]:
# Reclamações
# nome do arquivo
fname_reclamacoes_202101 = path_stage_reclamacoes + flist_raw_reclamacoes[0]
fname_reclamacoes_202102 = path_stage_reclamacoes + flist_raw_reclamacoes[1]
fname_reclamacoes_202103 = path_stage_reclamacoes + flist_raw_reclamacoes[2]
fname_reclamacoes_202104 = path_stage_reclamacoes + flist_raw_reclamacoes[3]
fname_reclamacoes_202201 = path_stage_reclamacoes + flist_raw_reclamacoes[4]
fname_reclamacoes_202203 = path_stage_reclamacoes + flist_raw_reclamacoes[6]
fname_reclamacoes_202204 = path_stage_reclamacoes + flist_raw_reclamacoes[7]

fname_reclamacoes_202101 = fname_reclamacoes_202101.replace('.csv', '_cleaned.csv')
fname_reclamacoes_202102 = fname_reclamacoes_202102.replace('.csv', '_cleaned.csv')
fname_reclamacoes_202103 = fname_reclamacoes_202103.replace('.csv', '_cleaned.csv')
fname_reclamacoes_202104 = fname_reclamacoes_202104.replace('.csv', '_cleaned.csv')
fname_reclamacoes_202201 = fname_reclamacoes_202201.replace('.csv', '_cleaned.csv')
fname_reclamacoes_202203 = fname_reclamacoes_202203.replace('.csv', '_cleaned.csv')
fname_reclamacoes_202204 = fname_reclamacoes_202204.replace('.csv', '_cleaned.csv')

fname_reclamacoes_202101 = fname_reclamacoes_202101.replace('\\', '/')
fname_reclamacoes_202102 = fname_reclamacoes_202102.replace('\\', '/')
fname_reclamacoes_202103 = fname_reclamacoes_202103.replace('\\', '/')
fname_reclamacoes_202104 = fname_reclamacoes_202104.replace('\\', '/')
fname_reclamacoes_202201 = fname_reclamacoes_202201.replace('\\', '/')
fname_reclamacoes_202203 = fname_reclamacoes_202203.replace('\\', '/')
fname_reclamacoes_202204 = fname_reclamacoes_202204.replace('\\', '/')

# write CSV
df_reclamacoes_202101.to_csv(fname_reclamacoes_202101)
df_reclamacoes_202102.to_csv(fname_reclamacoes_202102)
df_reclamacoes_202103.to_csv(fname_reclamacoes_202103)
df_reclamacoes_202104.to_csv(fname_reclamacoes_202104)
df_reclamacoes_202201.to_csv(fname_reclamacoes_202201)
df_reclamacoes_202203.to_csv(fname_reclamacoes_202203)
df_reclamacoes_202204.to_csv(fname_reclamacoes_202204)

print(fname_reclamacoes_202101)
print(fname_reclamacoes_202102)
print(fname_reclamacoes_202103)
print(fname_reclamacoes_202104)
print(fname_reclamacoes_202201)
print(fname_reclamacoes_202203)
print(fname_reclamacoes_202204)

c:/Users/Diane/git/grupo1-ingestao-dados/stage/reclamacoes/2021_tri_01_cleaned.csv
c:/Users/Diane/git/grupo1-ingestao-dados/stage/reclamacoes/2021_tri_02_cleaned.csv
c:/Users/Diane/git/grupo1-ingestao-dados/stage/reclamacoes/2021_tri_03_cleaned.csv
c:/Users/Diane/git/grupo1-ingestao-dados/stage/reclamacoes/2021_tri_04_cleaned.csv
c:/Users/Diane/git/grupo1-ingestao-dados/stage/reclamacoes/2022_tri_01_cleaned.csv
c:/Users/Diane/git/grupo1-ingestao-dados/stage/reclamacoes/2022_tri_03_cleaned.csv
c:/Users/Diane/git/grupo1-ingestao-dados/stage/reclamacoes/2022_tri_04_cleaned.csv
